# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
!pip install pyspark=="3.2.4"

In [2]:
import pandas as pd
import json

clickstream = pd.read_csv("clickstream.csv", delimiter="\t")

clickstream['is_error'] = clickstream['event_type'].str.contains('error', case=False, na=False)
error_timestamps = clickstream[clickstream['is_error']].groupby(['user_id', 'session_id'])['timestamp'].min().reset_index()
error_timestamps = error_timestamps.rename(columns={'timestamp': 'error_timestamp'})
clickstream = clickstream.merge(error_timestamps, on=['user_id', 'session_id'], how='left')
clickstream = clickstream[(clickstream['timestamp'] < clickstream['error_timestamp']) | clickstream['error_timestamp'].isna()]
clickstream = clickstream.sort_values(['user_id', 'session_id', 'timestamp'])

def collapse_route(pages):
    collapsed = [pages.iloc[0]]
    for page in pages[1:]:
        if page != collapsed[-1]:
            collapsed.append(page)
    return '-'.join(collapsed)

page_routes = clickstream.groupby(['user_id', 'session_id'])['event_page'].apply(collapse_route).reset_index()
page_routes = page_routes.rename(columns={'event_page': 'route'})

route_counts = page_routes['route'].value_counts().reset_index().head(10)
route_counts.columns = ['route', 'count']

result = {row["route"]: int(row['count']) for i, row in route_counts.iterrows()}

with open("result.json", "w") as f:
    json.dump(result, f, indent=4)

print(result)

{'main': 8184, 'main-archive': 1113, 'main-rabota': 1047, 'main-internet': 897, 'main-bonus': 870, 'main-news': 769, 'main-tariffs': 677, 'main-online': 587, 'main-vklad': 518, 'main-rabota-archive': 170}


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.window import Window

class ClickstreamAnalyzer:
    def __init__(self, input_path="hdfs:/data/clickstream.csv"):
        self.spark = SparkSession.builder.appName("ClickstreamAnalysis").getOrCreate()
        self.input_path = input_path

    def sql_solution(self):
        df = self.spark.read.option("delimiter", "\t").csv(self.input_path, header=True).withColumn("timestamp", F.col("timestamp").cast("int"))
        df.createOrReplaceTempView("clickstream")

        result = self.spark.sql("""
            WITH error_sessions AS (
                SELECT 
                    user_id,
                    session_id,
                    MIN(timestamp) AS error_timestamp
                FROM clickstream
                WHERE LOWER(event_type) LIKE '%error%'
                GROUP BY user_id, session_id
            ),
            filtered_clickstream AS (
                SELECT
                    c.user_id,
                    c.session_id,
                    c.event_page,
                    c.timestamp,
                    ROW_NUMBER() OVER (PARTITION BY c.user_id, c.session_id ORDER BY c.timestamp) AS rn,
                    LAG(c.event_page) OVER (PARTITION BY c.user_id, c.session_id ORDER BY c.timestamp) AS prev_page
                FROM clickstream c
                LEFT JOIN error_sessions e
                ON c.user_id = e.user_id AND c.session_id = e.session_id
                WHERE e.error_timestamp IS NULL OR c.timestamp < e.error_timestamp
            ),
            collapsed_routes AS (
                SELECT 
                    user_id,
                    session_id,
                    CONCAT_WS('-', COLLECT_LIST(event_page)) AS route
                FROM (
                    SELECT
                        user_id,
                        session_id,
                        event_page,
                        rn
                    FROM filtered_clickstream
                    WHERE event_page != prev_page OR prev_page IS NULL
                    ORDER BY user_id, session_id, rn  -- Ensure ordering for collect_list consistency
                ) AS unique_pages
                GROUP BY user_id, session_id
            )
            SELECT route, COUNT(*) AS count
            FROM collapsed_routes
            GROUP BY route
            ORDER BY count DESC
            LIMIT 30
        """)

        return result

    def rdd_solution(self):
        def parse_line(line):
            fields = line.split('\t')
            return (int(fields[0]), int(fields[1]), fields[2], fields[3], int(fields[4]))

        def filter_and_collapse(events):
            events = sorted(events, key=lambda x: x[4])
            error_time = next((t for (t, et) in [(e[4], e[2]) for e in events] if "error" in et), float('inf'))
            pages = [e[3] for e in events if e[4] < error_time]
            
            collapsed = []
            for page in pages:
                if not collapsed or page != collapsed[-1]:
                    collapsed.append(page)
            return '-'.join(collapsed) if collapsed else None

        result = (self.spark.sparkContext
                  .textFile(self.input_path)
                  .filter(lambda x: not x.startswith('user_id'))
                  .map(parse_line)
                  .map(lambda x: ((x[0], x[1]), x))
                  .groupByKey()
                  .mapValues(filter_and_collapse)
                  .filter(lambda x: x[1] is not None)
                  .map(lambda x: (x[1], 1))
                  .reduceByKey(lambda a, b: a + b)
                  .sortBy(lambda x: -x[1])
                  .take(30))

        return self.spark.createDataFrame(result, ["route", "count"])

    def df_solution(self):
        clickstream = (
            self.spark.read.option("delimiter", "\t")
            .option("header", True)
            .csv(self.input_path)
            .withColumn("user_id", F.col("user_id").cast("int"))
            .withColumn("session_id", F.col("session_id").cast("int"))
            .withColumn("timestamp", F.col("timestamp").cast("int"))
        )

        error_timestamps = (
            clickstream
            .filter(F.lower(F.col("event_type")).contains("error"))
            .groupBy("user_id", "session_id")
            .agg(F.min("timestamp").alias("error_timestamp"))
        )

        clickstream = (
            clickstream
            .join(error_timestamps, on=["user_id", "session_id"], how="left")
            .filter((F.col("timestamp") < F.col("error_timestamp")) | F.col("error_timestamp").isNull())
        )

        window_spec = Window.partitionBy("user_id", "session_id").orderBy("timestamp")
        clickstream = clickstream.withColumn("ordered_event_page", F.col("event_page"))

        clickstream = (
            clickstream
            .withColumn("prev_event_page", F.lag("ordered_event_page").over(window_spec))
            .withColumn("is_duplicate", F.when(F.col("ordered_event_page") == F.col("prev_event_page"), 1).otherwise(0))
            .filter(F.col("is_duplicate") == 0)
            .groupBy("user_id", "session_id")
            .agg(F.collect_list("ordered_event_page").alias("pages"))
            .withColumn("route", F.expr("concat_ws('-', pages)"))
            .select("user_id", "session_id", "route")
        )

        route_counts = (
            clickstream.groupBy("route")
            .count()
            .orderBy(F.desc("count"))
            .limit(30)
        )

        return route_counts

In [4]:
def compare_solutions(analyzer):
    implementations = {
        "RDD": analyzer.rdd_solution,
        "DataFrame": analyzer.df_solution,
        "SQL": analyzer.sql_solution,
    }
    
    results = {}
    for name, implementation in implementations.items():
        results[name] = implementation().toPandas()
    
    comparisons = [
        ("SQL", "RDD"),
        ("SQL", "DataFrame"),
        ("RDD", "DataFrame")
    ]

    for impl1, impl2 in comparisons:
        res1, res2 = list(results[impl1].itertuples(index=False)), list(results[impl2].itertuples(index=False))
        all_correct = True
        print(f"Comparing {impl1} to {impl2}")
        for i in range(30):
            if res1[i] != res2[i]:
                all_correct = False
                print(f"Mismatched result: {impl1} - {res1[i]} vs. {impl2} - {res2[i]}")
        if all_correct:
            print(f"{impl1} and {impl2} are identical")
    
    return results

In [5]:
analyzer = ClickstreamAnalyzer()
compare_solutions(analyzer)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-10-26 22:56:05,908 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


Comparing SQL to RDD
SQL and RDD are identical
Comparing SQL to DataFrame
SQL and DataFrame are identical
Comparing RDD to DataFrame
RDD and DataFrame are identical


{'RDD':                     route  count
 0                    main   8184
 1            main-archive   1113
 2             main-rabota   1047
 3           main-internet    897
 4              main-bonus    870
 5               main-news    769
 6            main-tariffs    677
 7             main-online    587
 8              main-vklad    518
 9     main-rabota-archive    170
 10    main-archive-rabota    167
 11     main-bonus-archive    143
 12      main-rabota-bonus    139
 13       main-news-rabota    135
 14      main-bonus-rabota    135
 15  main-archive-internet    132
 16       main-rabota-news    130
 17   main-internet-rabota    129
 18      main-archive-news    126
 19   main-rabota-internet    124
 20  main-internet-archive    123
 21     main-archive-bonus    117
 22    main-internet-bonus    115
 23  main-tariffs-internet    114
 24      main-news-archive    113
 25     main-news-internet    109
 26   main-archive-tariffs    104
 27     main-internet-news    103
 28   m